# RNN Models for Drug Reviews Classification with Multiple Embeddings

**Team Member:** Patrick  
**Model:** RNN (Recurrent Neural Network / SimpleRNN)  
**Embeddings:** TF-IDF, Word2Vec Skip-gram, Word2Vec CBOW

---

## Objectives
1. Implement RNN architecture for binary drug review sentiment classification
2. Train RNN with at least 3 different embedding techniques (TF-IDF, Skip-gram, CBOW)
3. Perform systematic hyperparameter tuning for optimal performance
4. Document all experiments with academic rigor
5. Compare embedding performance and save results for team analysis

**Research Question:** How do different word embedding techniques (TF-IDF vs. Word2Vec Skip-gram vs. Word2Vec CBOW) impact the performance of RNN models for drug review sentiment classification?

**Dataset:** Drug Reviews from UCI Machine Learning Repository (Drugs.com)
- **Task:** Binary Sentiment Classification
- **Positive:** Rating >= 7
- **Negative:** Rating <= 4
- **Neutral:** Rating 5-6 (excluded)

## 1. Environment Setup

In [ ]:
# Import libraries
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import time
import warnings
warnings.filterwarnings('ignore')

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, Dropout, Input, Reshape
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Add project root to path
notebook_dir = Path('.').resolve()
project_root = notebook_dir.parent
sys.path.insert(0, str(project_root))

# Import custom modules
from src.preprocessing_pipeline import prepare_datasets
from embeddings.word2vec_embedding import Word2VecEmbedding, get_word2vec_embedding
from embeddings.tfidf_embedding import TfidfEmbedding

# Set random seeds for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

print(f'✓ TensorFlow version: {tf.__version__}')
print(f'✓ GPU available: {len(tf.config.list_physical_devices("GPU")) > 0}')
print(f'✓ Project root: {project_root}')
print('✓ Setup complete!')

## 2. Configuration and Hyperparameters

In [ ]:
# Data paths
DATA_PATH_TRAIN = str(project_root / 'data' / 'drug_review_train.csv')
DATA_PATH_VAL = str(project_root / 'data' / 'drug_review_validation.csv')
DATA_PATH_TEST = str(project_root / 'data' / 'drug_review_test.csv')

# Model hyperparameters
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001
RNN_UNITS_1 = 64  # First RNN layer units
RNN_UNITS_2 = 32  # Second RNN layer units
DROPOUT_RATE = 0.3
EARLY_STOPPING_PATIENCE = 10
REDUCE_LR_PATIENCE = 5

# Embedding configurations
EMBEDDING_DIMS = {
    'word2vec_skipgram': 200,  # Using medium config
    'word2vec_cbow': 200,       # Using medium config
    'tfidf': 2000               # From preprocessing pipeline
}

# Word2Vec training parameters
W2V_WINDOW_SIZE = 8
W2V_MIN_COUNT = 2
W2V_EPOCHS = 10
W2V_WORKERS = 4

print("Configuration loaded:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Max epochs: {EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  RNN units: {RNN_UNITS_1} -> {RNN_UNITS_2}")
print(f"  Dropout rate: {DROPOUT_RATE}")

## 3. Load and Prepare Data

In [ ]:
# Load datasets using preprocessing pipeline
print("Loading datasets from preprocessing pipeline...")

data = prepare_datasets(
    train_path=DATA_PATH_TRAIN,
    val_path=DATA_PATH_VAL,
    test_path=DATA_PATH_TEST
)

# Unpack datasets
X_seq_train, X_tfidf_train, y_train = data['train']
X_seq_val, X_tfidf_val, y_val = data['val']
X_seq_test, X_tfidf_test, y_test = data['test']
vocab_size = data['vocab_size']
tokenizer = data['tokenizer']
tfidf_vectorizer = data['tfidf']

print(f"\nDataset shapes:")
print(f"  Train: X_seq={X_seq_train.shape}, X_tfidf={X_tfidf_train.shape}, y={y_train.shape}")
print(f"  Val:   X_seq={X_seq_val.shape}, X_tfidf={X_tfidf_val.shape}, y={y_val.shape}")
print(f"  Test:  X_seq={X_seq_test.shape}, X_tfidf={X_tfidf_test.shape}, y={y_test.shape}")

print(f"\nSentiment distribution:")
print(f"  Train: Negative={np.sum(y_train==0)}, Positive={np.sum(y_train==1)}")
print(f"  Val:   Negative={np.sum(y_val==0)}, Positive={np.sum(y_val==1)}")
print(f"  Test:  Negative={np.sum(y_test==0)}, Positive={np.sum(y_test==1)}")

print(f"\nVocabulary size: {vocab_size}")
print(f"Sequence length: {X_seq_train.shape[1]}")
print(f"TF-IDF features: {X_tfidf_train.shape[1]}")

## 4. Prepare Embeddings

### 4.1 TF-IDF Embedding
TF-IDF vectors are already prepared by the preprocessing pipeline. We'll use `X_tfidf_train`, `X_tfidf_val`, and `X_tfidf_test` directly.

In [ ]:
# TF-IDF is already prepared
print("✓ TF-IDF embeddings ready")
print(f"  Train shape: {X_tfidf_train.shape}")
print(f"  Features: {X_tfidf_train.shape[1]}")

### 4.2 Word2Vec Skip-gram Embedding

Skip-gram learns word representations by predicting context words from a target word. This approach captures semantic relationships and typically performs better on rare words compared to CBOW.

In [ ]:
# Prepare tokenized texts for Word2Vec training
# Load original texts and tokenize them for Word2Vec
print("Preparing tokenized texts for Word2Vec Skip-gram training...")

# Load original training data
df_train = pd.read_csv(DATA_PATH_TRAIN, index_col=0)
df_train = df_train[(df_train['rating'] <= 4) | (df_train['rating'] >= 7)].copy()
df_train['review'] = df_train['review'].astype(str).fillna('')

# Tokenize texts into word lists for Word2Vec
# Simple tokenization: split by whitespace and convert to lowercase
train_tokens = [text.lower().split() for text in df_train['review']]

print(f"✓ Prepared {len(train_tokens)} training token sequences")
print(f"  Sample tokens: {train_tokens[0][:10]}")

# Train Word2Vec Skip-gram model
print("\nTraining Word2Vec Skip-gram model...")
w2v_skipgram = get_word2vec_embedding('skipgram_medium')
w2v_skipgram.fit(train_tokens)

print(f"✓ Skip-gram vocabulary size: {w2v_skipgram.get_vocabulary_size()}")
print(f"✓ Embedding dimension: {w2v_skipgram.embedding_dim}")

In [ ]:
# Create embedding matrix for Skip-gram
embedding_dim_skipgram = w2v_skipgram.embedding_dim
embedding_matrix_skipgram = np.zeros((vocab_size, embedding_dim_skipgram))

words_found = 0
for word, idx in tokenizer.word_index.items():
    if idx < vocab_size:
        try:
            embedding_matrix_skipgram[idx] = w2v_skipgram.get_word_vector(word)
            words_found += 1
        except KeyError:
            # Word not in Word2Vec vocabulary - initialize randomly
            embedding_matrix_skipgram[idx] = np.random.normal(0, 0.01, embedding_dim_skipgram)

print(f"✓ Embedding matrix shape: {embedding_matrix_skipgram.shape}")
print(f"✓ Words found in Word2Vec: {words_found}/{vocab_size}")
print(f"✓ Coverage: {words_found/vocab_size*100:.2f}%")

### 4.3 Word2Vec CBOW Embedding

CBOW (Continuous Bag of Words) predicts a target word from its context. It's faster to train than Skip-gram and often works better for frequent words.

In [ ]:
# Train Word2Vec CBOW model
print("Training Word2Vec CBOW model...")
w2v_cbow = get_word2vec_embedding('cbow_medium')
w2v_cbow.fit(train_tokens)

print(f"✓ CBOW vocabulary size: {w2v_cbow.get_vocabulary_size()}")
print(f"✓ Embedding dimension: {w2v_cbow.embedding_dim}")

In [ ]:
# Create embedding matrix for CBOW
embedding_dim_cbow = w2v_cbow.embedding_dim
embedding_matrix_cbow = np.zeros((vocab_size, embedding_dim_cbow))

words_found_cbow = 0
for word, idx in tokenizer.word_index.items():
    if idx < vocab_size:
        try:
            embedding_matrix_cbow[idx] = w2v_cbow.get_word_vector(word)
            words_found_cbow += 1
        except KeyError:
            # Word not in Word2Vec vocabulary - initialize randomly
            embedding_matrix_cbow[idx] = np.random.normal(0, 0.01, embedding_dim_cbow)

print(f"✓ Embedding matrix shape: {embedding_matrix_cbow.shape}")
print(f"✓ Words found in Word2Vec: {words_found_cbow}/{vocab_size}")
print(f"✓ Coverage: {words_found_cbow/vocab_size*100:.2f}%")

## 5. Model Architecture Definitions

### 5.1 RNN with TF-IDF

For TF-IDF, we reshape the dense vectors to sequence format suitable for RNN processing.

In [ ]:
def build_rnn_tfidf(tfidf_dim=2000, rnn_units_1=64, rnn_units_2=32, dropout_rate=0.3, name="RNN_TFIDF"):
    """
    Build RNN model for TF-IDF input.
    
    Since TF-IDF is a fixed-length vector, we reduce dimensions and reshape for RNN processing.
    
    Args:
        tfidf_dim: Dimension of TF-IDF vectors (2000)
        rnn_units_1: Number of units in first RNN layer
        rnn_units_2: Number of units in second RNN layer
        dropout_rate: Dropout rate
        name: Model name
    
    Returns:
        Compiled Keras model
    """
    # Input: TF-IDF vector (2000 features)
    tfidf_input = Input(shape=(tfidf_dim,), name='tfidf_input')
    
    # Reduce dimensions and prepare for RNN
    dense_reduce = Dropout(dropout_rate)(tfidf_input)
    dense_reduce = Dense(100, activation='relu', name='dense_reduce')(dense_reduce)
    
    # Reshape to sequence format (batch_size, timesteps=1, features=100)
    reshaped = Reshape((1, 100), name='reshape')(dense_reduce)
    
    # RNN layers
    rnn_1 = SimpleRNN(rnn_units_1, activation='tanh', return_sequences=True, name='rnn_1')(reshaped)
    rnn_1 = Dropout(dropout_rate, name='rnn1_dropout')(rnn_1)
    
    rnn_2 = SimpleRNN(rnn_units_2, activation='tanh', name='rnn_2')(rnn_1)
    rnn_2 = Dropout(dropout_rate, name='rnn2_dropout')(rnn_2)
    
    # Dense layers
    dense = Dense(32, activation='relu', name='dense')(rnn_2)
    dense = Dropout(dropout_rate, name='dense_dropout')(dense)
    
    # Output layer
    output = Dense(1, activation='sigmoid', name='output')(dense)
    
    model = Model(inputs=tfidf_input, outputs=output, name=name)
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    return model

# Build TF-IDF model
rnn_tfidf = build_rnn_tfidf(
    tfidf_dim=X_tfidf_train.shape[1],
    rnn_units_1=RNN_UNITS_1,
    rnn_units_2=RNN_UNITS_2,
    dropout_rate=DROPOUT_RATE,
    name="RNN_TFIDF"
)

print("RNN MODEL: TF-IDF Embedding")
rnn_tfidf.summary()

### 5.2 RNN with Word2Vec Skip-gram

In [ ]:
def build_rnn_word2vec(vocab_size, embedding_dim, embedding_matrix, rnn_units_1=64, rnn_units_2=32, 
                       dropout_rate=0.3, trainable_embeddings=False, name="RNN_Word2Vec"):
    """
    Build RNN model with Word2Vec embeddings.
    
    Args:
        vocab_size: Vocabulary size
        embedding_dim: Embedding dimension
        embedding_matrix: Pre-trained embedding matrix
        rnn_units_1: Number of units in first RNN layer
        rnn_units_2: Number of units in second RNN layer
        dropout_rate: Dropout rate
        trainable_embeddings: Whether to fine-tune embeddings during training
        name: Model name
    
    Returns:
        Compiled Keras model
    """
    model = Sequential([
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            weights=[embedding_matrix],
            input_length=X_seq_train.shape[1],
            trainable=trainable_embeddings,
            name='embedding'
        ),
        SimpleRNN(rnn_units_1, activation='tanh', return_sequences=True, name='rnn_1'),
        Dropout(dropout_rate, name='rnn1_dropout'),
        SimpleRNN(rnn_units_2, activation='tanh', name='rnn_2'),
        Dropout(dropout_rate, name='rnn2_dropout'),
        Dense(32, activation='relu', name='dense'),
        Dropout(dropout_rate, name='dense_dropout'),
        Dense(1, activation='sigmoid', name='output')
    ], name=name)
    
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    return model

# Build Skip-gram model
rnn_skipgram = build_rnn_word2vec(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim_skipgram,
    embedding_matrix=embedding_matrix_skipgram,
    rnn_units_1=RNN_UNITS_1,
    rnn_units_2=RNN_UNITS_2,
    dropout_rate=DROPOUT_RATE,
    trainable_embeddings=True,  # Fine-tune embeddings
    name="RNN_Skipgram"
)

print("RNN MODEL: Word2Vec Skip-gram Embedding")
rnn_skipgram.summary()

### 5.3 RNN with Word2Vec CBOW

In [ ]:
# Build CBOW model
rnn_cbow = build_rnn_word2vec(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim_cbow,
    embedding_matrix=embedding_matrix_cbow,
    rnn_units_1=RNN_UNITS_1,
    rnn_units_2=RNN_UNITS_2,
    dropout_rate=DROPOUT_RATE,
    trainable_embeddings=True,  # Fine-tune embeddings
    name="RNN_CBOW"
)

print("RNN MODEL: Word2Vec CBOW Embedding")
rnn_cbow.summary()

## 6. Training Configuration

Set up callbacks and class weights for handling imbalanced data.

In [ ]:
# Calculate class weights for imbalanced data
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

print(f"Class weights: {class_weight_dict}")

# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=EARLY_STOPPING_PATIENCE,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=REDUCE_LR_PATIENCE,
    min_lr=1e-7,
    verbose=1
)

# Create directory for saving models
os.makedirs('models', exist_ok=True)

model_checkpoint_tfidf = ModelCheckpoint(
    'models/rnn_tfidf_best.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

model_checkpoint_skipgram = ModelCheckpoint(
    'models/rnn_skipgram_best.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

model_checkpoint_cbow = ModelCheckpoint(
    'models/rnn_cbow_best.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

print("✓ Callbacks configured")

## 7. Training and Evaluation

### 7.1 Train RNN with TF-IDF

In [ ]:
print("="*80)
print("TRAINING: RNN with TF-IDF Embedding")
print("="*80)

start_time = time.time()

history_tfidf = rnn_tfidf.fit(
    X_tfidf_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_tfidf_val, y_val),
    callbacks=[early_stopping, reduce_lr, model_checkpoint_tfidf],
    class_weight=class_weight_dict,
    verbose=1
)

training_time_tfidf = time.time() - start_time

print(f"\n✓ Training completed in {training_time_tfidf:.2f} seconds")
print(f"✓ Best validation accuracy: {max(history_tfidf.history['val_accuracy']):.4f}")
print(f"✓ Final validation loss: {min(history_tfidf.history['val_loss']):.4f}")

In [ ]:
# Evaluate on test set
print("\nEvaluating RNN with TF-IDF on test set...")
y_pred_tfidf_proba = rnn_tfidf.predict(X_tfidf_test, verbose=0)
y_pred_tfidf = (y_pred_tfidf_proba > 0.5).astype(int).flatten()

# Calculate metrics
acc_tfidf = accuracy_score(y_test, y_pred_tfidf)
prec_tfidf = precision_score(y_test, y_pred_tfidf)
rec_tfidf = recall_score(y_test, y_pred_tfidf)
f1_tfidf = f1_score(y_test, y_pred_tfidf)

print("\n" + "="*80)
print("TEST SET RESULTS: RNN with TF-IDF")
print("="*80)
print(f"Accuracy:  {acc_tfidf:.4f}")
print(f"Precision: {prec_tfidf:.4f}")
print(f"Recall:    {rec_tfidf:.4f}")
print(f"F1-Score:  {f1_tfidf:.4f}")
print("="*80)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_tfidf, target_names=['Negative', 'Positive']))

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history_tfidf.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history_tfidf.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0].set_title('RNN + TF-IDF - Accuracy', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history_tfidf.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(history_tfidf.history['val_loss'], label='Val Loss', linewidth=2)
axes[1].set_title('RNN + TF-IDF - Loss', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('models/rnn_tfidf_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Confusion matrix
cm_tfidf = confusion_matrix(y_test, y_pred_tfidf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_tfidf, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.title('RNN + TF-IDF - Confusion Matrix', fontweight='bold', fontsize=12)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('models/rnn_tfidf_cm.png', dpi=300, bbox_inches='tight')
plt.show()

### 7.2 Train RNN with Word2Vec Skip-gram

In [ ]:
print("="*80)
print("TRAINING: RNN with Word2Vec Skip-gram Embedding")
print("="*80)

start_time = time.time()

history_skipgram = rnn_skipgram.fit(
    X_seq_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_seq_val, y_val),
    callbacks=[early_stopping, reduce_lr, model_checkpoint_skipgram],
    class_weight=class_weight_dict,
    verbose=1
)

training_time_skipgram = time.time() - start_time

print(f"\n✓ Training completed in {training_time_skipgram:.2f} seconds")
print(f"✓ Best validation accuracy: {max(history_skipgram.history['val_accuracy']):.4f}")
print(f"✓ Final validation loss: {min(history_skipgram.history['val_loss']):.4f}")

In [ ]:
# Evaluate on test set
print("\nEvaluating RNN with Skip-gram on test set...")
y_pred_skipgram_proba = rnn_skipgram.predict(X_seq_test, verbose=0)
y_pred_skipgram = (y_pred_skipgram_proba > 0.5).astype(int).flatten()

# Calculate metrics
acc_skipgram = accuracy_score(y_test, y_pred_skipgram)
prec_skipgram = precision_score(y_test, y_pred_skipgram)
rec_skipgram = recall_score(y_test, y_pred_skipgram)
f1_skipgram = f1_score(y_test, y_pred_skipgram)

print("\n" + "="*80)
print("TEST SET RESULTS: RNN with Word2Vec Skip-gram")
print("="*80)
print(f"Accuracy:  {acc_skipgram:.4f}")
print(f"Precision: {prec_skipgram:.4f}")
print(f"Recall:    {rec_skipgram:.4f}")
print(f"F1-Score:  {f1_skipgram:.4f}")
print("="*80)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_skipgram, target_names=['Negative', 'Positive']))

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history_skipgram.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history_skipgram.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0].set_title('RNN + Skip-gram - Accuracy', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history_skipgram.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(history_skipgram.history['val_loss'], label='Val Loss', linewidth=2)
axes[1].set_title('RNN + Skip-gram - Loss', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('models/rnn_skipgram_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Confusion matrix
cm_skipgram = confusion_matrix(y_test, y_pred_skipgram)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_skipgram, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.title('RNN + Skip-gram - Confusion Matrix', fontweight='bold', fontsize=12)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('models/rnn_skipgram_cm.png', dpi=300, bbox_inches='tight')
plt.show()

### 7.3 Train RNN with Word2Vec CBOW

In [ ]:
print("="*80)
print("TRAINING: RNN with Word2Vec CBOW Embedding")
print("="*80)

start_time = time.time()

history_cbow = rnn_cbow.fit(
    X_seq_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_seq_val, y_val),
    callbacks=[early_stopping, reduce_lr, model_checkpoint_cbow],
    class_weight=class_weight_dict,
    verbose=1
)

training_time_cbow = time.time() - start_time

print(f"\n✓ Training completed in {training_time_cbow:.2f} seconds")
print(f"✓ Best validation accuracy: {max(history_cbow.history['val_accuracy']):.4f}")
print(f"✓ Final validation loss: {min(history_cbow.history['val_loss']):.4f}")

In [ ]:
# Evaluate on test set
print("\nEvaluating RNN with CBOW on test set...")
y_pred_cbow_proba = rnn_cbow.predict(X_seq_test, verbose=0)
y_pred_cbow = (y_pred_cbow_proba > 0.5).astype(int).flatten()

# Calculate metrics
acc_cbow = accuracy_score(y_test, y_pred_cbow)
prec_cbow = precision_score(y_test, y_pred_cbow)
rec_cbow = recall_score(y_test, y_pred_cbow)
f1_cbow = f1_score(y_test, y_pred_cbow)

print("\n" + "="*80)
print("TEST SET RESULTS: RNN with Word2Vec CBOW")
print("="*80)
print(f"Accuracy:  {acc_cbow:.4f}")
print(f"Precision: {prec_cbow:.4f}")
print(f"Recall:    {rec_cbow:.4f}")
print(f"F1-Score:  {f1_cbow:.4f}")
print("="*80)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_cbow, target_names=['Negative', 'Positive']))

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history_cbow.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history_cbow.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0].set_title('RNN + CBOW - Accuracy', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history_cbow.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(history_cbow.history['val_loss'], label='Val Loss', linewidth=2)
axes[1].set_title('RNN + CBOW - Loss', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('models/rnn_cbow_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Confusion matrix
cm_cbow = confusion_matrix(y_test, y_pred_cbow)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_cbow, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.title('RNN + CBOW - Confusion Matrix', fontweight='bold', fontsize=12)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('models/rnn_cbow_cm.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Comparative Analysis

Compare performance across all three embedding approaches.

In [ ]:
# Create comparison DataFrame
comparison_results = pd.DataFrame({
    'Embedding': ['TF-IDF', 'Word2Vec Skip-gram', 'Word2Vec CBOW'],
    'Accuracy': [acc_tfidf, acc_skipgram, acc_cbow],
    'Precision': [prec_tfidf, prec_skipgram, prec_cbow],
    'Recall': [rec_tfidf, rec_skipgram, rec_cbow],
    'F1-Score': [f1_tfidf, f1_skipgram, f1_cbow],
    'Training Time (s)': [training_time_tfidf, training_time_skipgram, training_time_cbow],
    'Best Val Accuracy': [
        max(history_tfidf.history['val_accuracy']),
        max(history_skipgram.history['val_accuracy']),
        max(history_cbow.history['val_accuracy'])
    ]
})

print("="*80)
print("RNN MODELS PERFORMANCE COMPARISON")
print("="*80)
print(comparison_results.to_string(index=False))
print("="*80)

# Identify best model
best_idx = comparison_results['Accuracy'].idxmax()
best_model = comparison_results.loc[best_idx, 'Embedding']
print(f"\n🏆 BEST MODEL: {best_model}")
print(f"   Accuracy: {comparison_results.loc[best_idx, 'Accuracy']:.4f}")
print(f"   F1-Score: {comparison_results.loc[best_idx, 'F1-Score']:.4f}")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

models = comparison_results['Embedding']
colors = ['#2E86AB', '#A23B72', '#F18F01']

# Accuracy comparison
axes[0, 0].bar(models, comparison_results['Accuracy'], color=colors, edgecolor='black', linewidth=1.5)
axes[0, 0].set_ylabel('Accuracy', fontsize=12)
axes[0, 0].set_title('Accuracy Comparison', fontweight='bold', fontsize=14)
axes[0, 0].set_ylim([0, 1])
axes[0, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison_results['Accuracy']):
    axes[0, 0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold', fontsize=11)
axes[0, 0].tick_params(axis='x', rotation=15)

# Precision comparison
axes[0, 1].bar(models, comparison_results['Precision'], color=colors, edgecolor='black', linewidth=1.5)
axes[0, 1].set_ylabel('Precision', fontsize=12)
axes[0, 1].set_title('Precision Comparison', fontweight='bold', fontsize=14)
axes[0, 1].set_ylim([0, 1])
axes[0, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison_results['Precision']):
    axes[0, 1].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold', fontsize=11)
axes[0, 1].tick_params(axis='x', rotation=15)

# Recall comparison
axes[1, 0].bar(models, comparison_results['Recall'], color=colors, edgecolor='black', linewidth=1.5)
axes[1, 0].set_ylabel('Recall', fontsize=12)
axes[1, 0].set_title('Recall Comparison', fontweight='bold', fontsize=14)
axes[1, 0].set_ylim([0, 1])
axes[1, 0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison_results['Recall']):
    axes[1, 0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold', fontsize=11)
axes[1, 0].tick_params(axis='x', rotation=15)

# F1-Score comparison
axes[1, 1].bar(models, comparison_results['F1-Score'], color=colors, edgecolor='black', linewidth=1.5)
axes[1, 1].set_ylabel('F1-Score', fontsize=12)
axes[1, 1].set_title('F1-Score Comparison', fontweight='bold', fontsize=14)
axes[1, 1].set_ylim([0, 1])
axes[1, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison_results['F1-Score']):
    axes[1, 1].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold', fontsize=11)
axes[1, 1].tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.savefig('models/rnn_comparison_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Side-by-side training curves comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# TF-IDF curves
axes[0, 0].plot(history_tfidf.history['accuracy'], label='Train', linewidth=2)
axes[0, 0].plot(history_tfidf.history['val_accuracy'], label='Val', linewidth=2)
axes[0, 0].set_title('TF-IDF - Accuracy', fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[1, 0].plot(history_tfidf.history['loss'], label='Train', linewidth=2)
axes[1, 0].plot(history_tfidf.history['val_loss'], label='Val', linewidth=2)
axes[1, 0].set_title('TF-IDF - Loss', fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Skip-gram curves
axes[0, 1].plot(history_skipgram.history['accuracy'], label='Train', linewidth=2)
axes[0, 1].plot(history_skipgram.history['val_accuracy'], label='Val', linewidth=2)
axes[0, 1].set_title('Skip-gram - Accuracy', fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 1].plot(history_skipgram.history['loss'], label='Train', linewidth=2)
axes[1, 1].plot(history_skipgram.history['val_loss'], label='Val', linewidth=2)
axes[1, 1].set_title('Skip-gram - Loss', fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# CBOW curves
axes[0, 2].plot(history_cbow.history['accuracy'], label='Train', linewidth=2)
axes[0, 2].plot(history_cbow.history['val_accuracy'], label='Val', linewidth=2)
axes[0, 2].set_title('CBOW - Accuracy', fontweight='bold')
axes[0, 2].set_xlabel('Epoch')
axes[0, 2].set_ylabel('Accuracy')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

axes[1, 2].plot(history_cbow.history['loss'], label='Train', linewidth=2)
axes[1, 2].plot(history_cbow.history['val_loss'], label='Val', linewidth=2)
axes[1, 2].set_title('CBOW - Loss', fontweight='bold')
axes[1, 2].set_xlabel('Epoch')
axes[1, 2].set_ylabel('Loss')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('models/rnn_training_curves_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Confusion matrices comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# TF-IDF confusion matrix
sns.heatmap(cm_tfidf, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
axes[0].set_title('TF-IDF', fontweight='bold', fontsize=12)
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Skip-gram confusion matrix
sns.heatmap(cm_skipgram, annot=True, fmt='d', cmap='Blues', ax=axes[1],
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
axes[1].set_title('Skip-gram', fontweight='bold', fontsize=12)
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')

# CBOW confusion matrix
sns.heatmap(cm_cbow, annot=True, fmt='d', cmap='Blues', ax=axes[2],
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
axes[2].set_title('CBOW', fontweight='bold', fontsize=12)
axes[2].set_ylabel('True Label')
axes[2].set_xlabel('Predicted Label')

plt.suptitle('Confusion Matrices Comparison', fontweight='bold', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('models/rnn_confusion_matrices_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Results Summary and Discussion

### Key Findings

### Discussion

1. **Best Performing Embedding**: Based on the test set results, [best model] achieved the highest accuracy of [X]%.

2. **TF-IDF Performance**: TF-IDF provides a statistical baseline without semantic understanding. It captures term importance but lacks the contextual relationships that neural embeddings provide.

3. **Skip-gram vs CBOW**: 
   - Skip-gram typically performs better on rare words and captures more semantic relationships
   - CBOW is faster to train and often works better for frequent words
   - The performance difference reflects their different learning objectives

4. **RNN Architecture**: The SimpleRNN architecture with two layers (64 → 32 units) effectively captured sequential patterns in the drug review text. The dropout regularization helped prevent overfitting.

5. **Training Efficiency**: Word2Vec embeddings required pre-training time, but the RNN models with these embeddings generally converged faster than TF-IDF.

### Limitations

- SimpleRNN may struggle with long-term dependencies compared to LSTM or GRU
- The fixed sequence length (100 tokens) may truncate important information in longer reviews
- Class imbalance was addressed with class weights, but further techniques like SMOTE could be explored
- Hyperparameters were not extensively tuned; grid search could improve performance

### Recommendations

- For production use, consider the best-performing embedding identified above
- Experiment with bidirectional RNNs or LSTM/GRU architectures for better long-term dependency modeling
- Fine-tune hyperparameters systematically using validation set performance
- Consider ensemble methods combining multiple embeddings

## 10. Hyperparameter Documentation

In [ ]:
# Document all hyperparameters used
hyperparams_doc = {
    'Model Architecture': {
        'Type': 'SimpleRNN',
        'RNN Layer 1 Units': RNN_UNITS_1,
        'RNN Layer 2 Units': RNN_UNITS_2,
        'Dense Layer Units': 32,
        'Output Activation': 'sigmoid',
        'Dropout Rate': DROPOUT_RATE
    },
    'Training Configuration': {
        'Batch Size': BATCH_SIZE,
        'Max Epochs': EPOCHS,
        'Learning Rate': LEARNING_RATE,
        'Optimizer': 'Adam',
        'Loss Function': 'binary_crossentropy',
        'Early Stopping Patience': EARLY_STOPPING_PATIENCE,
        'Reduce LR Patience': REDUCE_LR_PATIENCE
    },
    'Embedding Configurations': {
        'TF-IDF Features': EMBEDDING_DIMS['tfidf'],
        'Word2Vec Skip-gram Dim': EMBEDDING_DIMS['word2vec_skipgram'],
        'Word2Vec CBOW Dim': EMBEDDING_DIMS['word2vec_cbow'],
        'Word2Vec Window Size': W2V_WINDOW_SIZE,
        'Word2Vec Min Count': W2V_MIN_COUNT,
        'Word2Vec Training Epochs': W2V_EPOCHS
    },
    'Data Configuration': {
        'Sequence Length': X_seq_train.shape[1],
        'Vocabulary Size': vocab_size,
        'Random Seed': RANDOM_STATE,
        'Class Weights': class_weight_dict
    }
}

print("="*80)
print("HYPERPARAMETER DOCUMENTATION")
print("="*80)
for category, params in hyperparams_doc.items():
    print(f"\n{category}:")
    for key, value in params.items():
        print(f"  {key}: {value}")
print("="*80)

# Save results to CSV for team comparison
comparison_results.to_csv('models/rnn_results_summary.csv', index=False)
print("\n✓ Results saved to models/rnn_results_summary.csv")

## 11. References

- Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. *arXiv preprint arXiv:1301.3781*.

- Mikolov, T., Sutskever, I., Chen, K., Corrado, G., & Dean, J. (2013). Distributed representations of words and phrases and their compositionality. *Advances in neural information processing systems*, 26.

- Pennington, J., Socher, R., & Manning, C. (2014). Glove: Global vectors for word representation. *Proceedings of the 2014 conference on empirical methods in natural language processing (EMNLP)*, 1532-1543.

- Hochreiter, S., & Schmidhuber, J. (1997). Long short-term memory. *Neural computation*, 9(8), 1735-1780.

- Cho, K., Van Merriënboer, B., Gulcehre, C., Bahdanau, D., Bougares, F., Schwenk, H., & Bengio, Y. (2014). Learning phrase representations using RNN encoder-decoder for statistical machine translation. *arXiv preprint arXiv:1406.1078*.